In [411]:
import pandas as pd
import numpy as np

In [408]:
np.random.seed(0)
df2 = pd.DataFrame(np.random.randn(10,4), columns=['A','B','C','D'])
df2.style

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [406]:
s2 = df2.style.set_caption('Датасет со случайными числами')
s2

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


Style фукнции должны возвращать строку с одним или более стилями пар CSS <атрибут>:<значение>, разделенных точкой с запятой

* Styler.applymap(func) для поэлементных стилей
* Styler.apply(func, axis=0) для столбцовых стилей
* Styler.apply(func, axis=1) для колоночных стилей
* Styler.apply(func, axis=None) на уровне таблицы

И что очень важно, входные и выходные формы func должны совпадать. Если x является входом, тогда func(x).shape == x.shape

In [269]:
def style_negative(v, props=''):
    return props if v < 0 else None

s2 = s2.applymap(style_negative, props='color:red')
s2

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [270]:
s2 = s2.applymap(lambda v: 'opacity: 50%;' if (abs(v) < 0.3) else None)
s2

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [301]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

s2.apply(highlight_max, props='color:white;background-color:red', axis=None)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [304]:
s2.apply(highlight_max, props='color:white;background-color:darkblue', axis=0)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [309]:
s2.apply(highlight_max, props='color:white;background-color:green', axis=1)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [310]:
df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
df.style

In [311]:
s = df.style.format('{:.0f}').hide([('Random', 'Tumour'), ('Random', 'Non-Tumour')], axis="columns")
s

In [323]:
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#b81f9c'),('color','lightblue'),('font-weight',800)]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #4a72f6; color: white;'
}
s.set_table_styles([cell_hover, index_names, headers])

In [324]:
s.set_table_styles([
    {'selector': 'th.col_heading', 'props': 'text-align: center;'},
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.5em;'},
    {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
], overwrite=False)

In [ ]:
s.to_html().split('\n')

In [357]:
np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[0, 2] = np.nan
df.iloc[0, 0] = 10000

In [358]:
df.style

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


Определенные классы CSS прикреплены к ячейкам.

* Имена индексов и столбцов включают `index_name` и `level<k>`, где `k` — это его уровень в MultiIndex.
* Index label ячейки включают
  * row_heading
  * row<n> где `n` цифровая позиция строки
  * level<k> где `k` уровень в Мультииндексе
* Column label ячейки включают
  * col_heading
  * col<n> где `n` цифровая позиция колонки
  * level<k> где `k` уровень в Мультииндексе
* Пустые ячейки включают `blank`
* Ячейки с данными включают `data`

In [359]:
df.style.set_table_styles([{'selector':'.blank','props':'background-color:lightblue;'}])

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [383]:

def highlight_max(data, backcolor='yellow', color='black'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}; color: {}; font-weight: 600'.format(backcolor, color)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [361]:
def mid(s, props=''):
    return np.where((s == 5) | (s =='C'), props, '')


In [362]:
def col_high(_):
  if (_ in df.columns.values) & (_== 'E'):
    return 'color:white;background-color:red; font-weight:bold'
  elif isinstance(_, float):
    return np.array('color:white;background-color:red;font-weight:bold')
  else:
    return ''

In [363]:
def smartRound(val):
  if val > 1:
    return f'{val:.3f}'
  elif abs(val) > .3:
    return f'{val:.1f}'
  else:
    return 'Очень маленькая величина'

In [385]:
df.style.apply(highlight_max, backcolor='darkorange', color = 'black',axis=None)

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [390]:
df.style.apply(highlight_max, backcolor='darkorange', color='blue', axis=0)

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [391]:
df.style.highlight_null(color='red')

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [393]:
df.style.highlight_null(color='red')\
  .highlight_max(color='green',axis=0)

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [394]:
df.style.highlight_null(color='red')\
  .highlight_max(color='green',axis=0)\
  .highlight_between(left=0, right=1, props='color: yellow; background-color: #4a72f6; font-weight:bold', subset = ['B','C'])

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [396]:
df.style.highlight_null(color='red')\
  .highlight_max(color='green',axis=0)\
  .highlight_between(left=0, right=1, props='color: yellow; background-color: #4a72f6; font-weight:bold', subset = ['B','C'])\
  .applymap_index(mid,axis = 0,  props='color:white;background-color:darkblue')\
  .applymap_index(mid,axis = 1,  props='color:white;background-color:darkblue')

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [397]:
df.style.highlight_null(color='red')\
  .highlight_max(color='green',axis=0)\
  .highlight_between(left=0, right=1, props='color: yellow; background-color: #4a72f6; font-weight:bold', subset = ['B','C'])\
  .applymap_index(mid,axis = 0,  props='color:white;background-color:darkblue')\
  .applymap_index(mid,axis = 1,  props='color:white;background-color:darkblue')\
  .applymap(col_high, subset=['E'])

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [400]:
df.style.highlight_null(color='red')\
  .highlight_max(color='green',axis=0)\
  .highlight_between(left=0, right=1, props='color: yellow; background-color: #4a72f6; font-weight:bold', subset = ['B','C'])\
  .applymap_index(mid,axis = 0,  props='color:white;background-color:darkblue')\
  .applymap_index(mid,axis = 1,  props='color:white;background-color:darkblue')\
  .applymap(col_high, subset=['E'])\
  .applymap_index(col_high, axis = 1)

,A,B,C,D,E
0,10000.000000,1.329212,nan,-0.316280,-0.990810
1,2.000000,-1.070816,-1.438713,0.564417,0.295722
2,3.000000,-1.626404,0.219565,0.678805,1.889273
3,4.000000,0.961538,0.104011,-0.481165,0.850229
4,5.000000,1.453425,1.057737,0.165562,0.515018
5,6.000000,-1.336936,0.562861,1.392855,-0.063328
6,7.000000,0.121668,1.207603,-0.002040,1.627796
7,8.000000,0.354493,1.037528,-0.385684,0.519818
8,9.000000,1.686583,-1.325963,1.428984,-2.089354
9,10.000000,-0.129820,0.631523,-0.586538,0.290720


In [399]:
df.style.highlight_null(color='red')\
  .highlight_max(color='green',axis=0)\
  .highlight_between(left=0, right=1, props='color: yellow; background-color: #4a72f6; font-weight:bold', subset = ['B','C'])\
  .applymap_index(mid,axis = 0,  props='color:white;background-color:darkblue')\
  .applymap_index(mid,axis = 1,  props='color:white;background-color:darkblue')\
  .applymap(col_high, subset=['E'])\
  .applymap_index(col_high, axis = 1)\
  .format({'A': '{:_.2f}', 'B': '£ {:.1f}', 'D':smartRound}, na_rep='DATA MISSING')


,A,B,C,D,E
0,10_000.00,£ 1.3,DATA MISSING,-0.3,-0.990810
1,2.00,£ -1.1,-1.438713,0.6,0.295722
2,3.00,£ -1.6,0.219565,0.7,1.889273
3,4.00,£ 1.0,0.104011,-0.5,0.850229
4,5.00,£ 1.5,1.057737,Очень маленькая величина,0.515018
5,6.00,£ -1.3,0.562861,1.393,-0.063328
6,7.00,£ 0.1,1.207603,Очень маленькая величина,1.627796
7,8.00,£ 0.4,1.037528,-0.4,0.519818
8,9.00,£ 1.7,-1.325963,1.429,-2.089354
9,10.00,£ -0.1,0.631523,-0.6,0.290720


In [401]:
df.style.highlight_null(color='red')\
  .highlight_max(color='green',axis=0)\
  .highlight_between(left=0, right=1, props='color: yellow; background-color: #4a72f6; font-weight:bold', subset = ['B','C'])\
  .applymap_index(mid,axis = 0,  props='color:white;background-color:darkblue')\
  .applymap_index(mid,axis = 1,  props='color:white;background-color:darkblue')\
  .applymap(col_high, subset=['E'])\
  .applymap_index(col_high, axis = 1)\
  .format({'A': '{:_.2f}', 'B': '£ {:.1f}', 'D':smartRound}, na_rep='DATA MISSING')

In [201]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "4pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

import seaborn as sns
np.random.seed(25)
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)
bigdf = pd.DataFrame(np.random.randn(20, 25)).cumsum()



bigdf.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '1pt'})\
    .set_caption("Подведите курсор для увеличения")\
    .set_table_styles(magnify()).format("{:.2%}")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,22.83%,102.69%,-83.96%,-59.12%,-95.69%,-22.23%,-61.99%,183.79%,-205.32%,86.86%,-92.07%,-23.23%,215.30%,-133.47%,7.64%,-124.61%,120.23%,-104.99%,105.66%,-41.97%,229.48%,-259.45%,282.28%,68.09%,-157.77%
1,-174.80%,156.02%,-113.05%,-110.47%,102.57%,0.37%,-245.98%,344.56%,-166.49%,126.83%,-51.53%,-1.53%,151.95%,-108.80%,-186.32%,-113.20%,-68.31%,-80.69%,35.11%,-5.51%,179.19%,-282.02%,225.72%,78.43%,44.07%
2,-65.37%,322.27%,-175.79%,51.65%,220.39%,-37.12%,-300.41%,373.33%,-187.08%,245.83%,21.37%,-23.74%,-10.32%,-77.55%,-302.36%,-81.85%,-21.11%,-22.90%,85.65%,-68.15%,144.55%,-488.62%,302.62%,191.32%,60.76%
3,-162.10%,371.47%,-230.88%,43.18%,417.14%,-43.39%,-385.53%,415.99%,-214.83%,108.04%,11.85%,59.63%,-88.79%,27.06%,-366.94%,-271.00%,-30.84%,-158.79%,135.48%,-182.89%,90.90%,-580.27%,281.40%,210.60%,28.50%
4,-334.86%,447.87%,-186.35%,-170.38%,519.18%,-102.13%,-380.72%,472.00%,-72.41%,107.72%,-17.93%,82.90%,-21.60%,-107.56%,-427.11%,-287.92%,-96.68%,-178.39%,153.24%,-179.66%,221.23%,-634.22%,334.39%,248.88%,208.56%
5,-83.50%,423.37%,-165.47%,-200.46%,534.48%,-99.03%,-413.22%,394.28%,-106.16%,-94.33%,124.05%,8.71%,-177.59%,-10.93%,-445.34%,-85.17%,-205.71%,-135.39%,80.14%,-163.24%,153.90%,-651.08%,280.21%,213.63%,377.44%
6,-74.27%,535.47%,-210.59%,-113.26%,420.27%,-185.00%,-320.15%,376.48%,-322.00%,-123.26%,33.57%,57.47%,-181.79%,54.00%,-443.43%,-182.95%,-403.00%,-262.00%,-19.95%,-468.23%,193.38%,-845.82%,333.54%,251.55%,581.44%
7,-43.54%,468.55%,-230.25%,-20.97%,592.99%,-262.50%,-183.48%,545.87%,-450.28%,-316.48%,-172.78%,17.92%,10.78%,3.63%,-599.26%,-44.95%,-619.93%,-388.97%,70.58%,-394.51%,67.34%,-725.66%,296.80%,339.43%,665.85%
8,91.64%,580.14%,-333.27%,-65.40%,598.73%,-318.78%,-183.03%,563.23%,-353.21%,-129.76%,-160.79%,81.98%,-244.57%,-40.36%,-605.55%,-51.66%,-659.59%,-348.41%,-4.34%,-459.91%,50.63%,-584.62%,322.72%,240.49%,507.55%
9,37.66%,553.64%,-448.92%,-80.14%,705.04%,-263.85%,-44.30%,534.61%,-195.85%,-33.46%,-80.09%,25.71%,-336.91%,-81.79%,-605.17%,-261.32%,-845.46%,-445.24%,41.35%,-470.97%,189.22%,-692.89%,213.77%,300.05%,515.69%
